<a href="https://colab.research.google.com/github/tejvapraj/Machine_learning_model/blob/main/Final_P1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import
from google.colab import drive
import pandas as pd
import regex as re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
drive.mount('/content/cdrive')

Mounted at /content/cdrive


In [178]:
# Load the dataset
Data= pd.read_excel(r'/content/cdrive/My Drive/Prediction of Placement Status Data/Final Lead Data.xlsx')

In [179]:
Data.head()

,ID,First Name,Email,Gender,City,Created,Position,New College Name,Colleges,Academic Year,Branch/ Specialisation,Other Branch,What is your current academic year?,Company Name/ College Name,Would you like to know more about us and our programs?,Are you interested in knowing more about our events?,Have you recommended Cloud Counselage to anyone?,How did you come to know about this event?
0,68112,ANIKET,aniket@xyz.com,NaN,NaN,04/27/2022 01:41:38 pm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,yes,no,NaN
1,68110,Dhanshree,dhanshree@xyz.com,NaN,NaN,04/22/2022 04:08:38 pm,NaN,Lords Universal College,NaN,NaN,NaN,NaN,NaN,NaN,yes,yes,no,NaN
2,68108,Dhiraj,dhiraj@xyz.com,NaN,NaN,04/16/2022 10:31:59 pm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,yes,no,NaN
3,68106,Pooja,pooja@xyz.com,NaN,NaN,04/13/2022 10:05:15 pm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,yes,no,NaN
4,68090,Aayush,aayush@xyz.com,NaN,NaN,03/26/2022 07:02:48 pm,NaN,B.k Birla college,NaN,NaN,NaN,NaN,NaN,NaN,yes,yes,no,NaN


In [180]:
# Handling Missing Values

median_academic_year = Data['Academic Year'].median()
Data['Academic Year'].fillna(median_academic_year, inplace=True)
# Handle missing values first
Data['What is your current academic year?'].fillna('Unknown', inplace=True)

# Then, handle string values using one-hot encoding
Data_encoded = pd.get_dummies(Data, columns=['What is your current academic year?'], dummy_na=True)

from sklearn.preprocessing import LabelEncoder
le_academic_year = LabelEncoder()
Data['Academic Year'] = le_academic_year.fit_transform(Data['Academic Year'])

le_current_academic_year = LabelEncoder()
Data['What is your current academic year?'] = le_academic_year.fit_transform(Data['What is your current academic year?'])

Data["Created"] = pd.to_datetime(Data["Created"], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')

In [184]:
def calculate_expected_graduation_year(row):
    current_academic_year = row["What is your current academic year?"]
    academic_year = row["Academic Year"]
    branch_specialisation = row["Branch/ Specialisation"]

    if current_academic_year == "Final Year":
        try:
            current_academic_year = int(academic_year)
        except ValueError:
            current_academic_year = -1
    else:
        try:
            current_academic_year = int(current_academic_year)
        except ValueError:
            current_academic_year = -1

    if current_academic_year != -1 and academic_year != -1:
        if current_academic_year == 1:
            return academic_year + 3
        elif current_academic_year == 2:
            return academic_year + 2
        elif current_academic_year == 3:
            return academic_year + 1
        elif current_academic_year == 4:
            if branch_specialisation == "Computer Science":
                return academic_year + 2        # Add two more years for Computer Science
            return academic_year + 1            # Otherwise, it's a regular 4-year program
                                                # If the current academic year is not 1, 2, 3, or 4
        else:
            return -1

# Assuming your DataFrame is called 'Data'
Data['Calculated Expected Graduation Year'] = Data.apply(calculate_expected_graduation_year, axis=1)

# Assuming Data is your DataFrame
Data["Expected Graduation Year"] = Data.apply(calculate_expected_graduation_year, axis=1)
# Print the first few rows of the DataFrame to check the results
import pandas as pd

# Assuming df is your DataFrame
Data['Expected Graduation Year'].replace(-1, 0, inplace=True)

print(Data.head())


      ID First Name              Email Gender City             Created  \
0  68112     ANIKET     aniket@xyz.com    NaN  NaN 2022-04-27 13:41:38   
1  68110  Dhanshree  dhanshree@xyz.com    NaN  NaN 2022-04-22 16:08:38   
2  68108     Dhiraj     dhiraj@xyz.com    NaN  NaN 2022-04-16 22:31:59   
3  68106      Pooja      pooja@xyz.com    NaN  NaN 2022-04-13 22:05:15   
4  68090     Aayush     aayush@xyz.com    NaN  NaN 2022-03-26 19:02:48   

  Position         New College Name Colleges  Academic Year  \
0      NaN                      NaN      NaN              2   
1      NaN  Lords Universal College      NaN              2   
2      NaN                      NaN      NaN              2   
3      NaN                      NaN      NaN              2   
4      NaN        B.k Birla college      NaN              2   

  Branch/ Specialisation Other Branch  What is your current academic year?  \
0                    NaN          NaN                                    4   
1                   

In [185]:
# Assuming your DataFrame is called 'Data'

# Create a boolean mask to identify rows where the values don't match
mask = Data['Expected Graduation Year'] != Data['Calculated Expected Graduation Year']

# Filter the DataFrame to show only the rows where the values don't match
mismatched_rows = Data[mask]

# Print or inspect the mismatched rows
print(mismatched_rows)



         ID  First Name               Email  Gender City             Created  \
2298  38958      Dennis      dennis@xyz.com    Male  NaN 2020-06-15 12:28:06   
2299  38956    Beatrice    beatrice@xyz.com  Female  NaN 2020-06-15 00:43:36   
2311  38932     Kendall     kendall@xyz.com  Female  NaN 2020-06-13 10:36:47   
3073  35936      Nayana      nayana@xyz.com  Female  NaN 2020-03-28 13:45:00   
3130  35760       Tejas       tejas@xyz.com  Female  NaN 2020-03-26 10:51:03   
3136  35748  Bhagyashri  bhagyashri@xyz.com  Female  NaN 2020-03-25 23:52:44   
3572  33518       Sahil       sahil@xyz.com    Male  NaN 2020-02-29 22:02:19   
3584  33488        Yash        yash@xyz.com    Male  NaN 2020-02-29 15:13:46   
3676  33304       Aryan       aryan@xyz.com    Male  NaN 2020-02-27 21:07:31   
3824  33008        Amit        amit@xyz.com  Female  NaN 2020-02-26 20:29:02   
4325  32000     Manohar     manohar@xyz.com    Male  NaN 2020-02-19 12:02:52   

     Position                          

In [186]:
print(Data["What is your current academic year?"].unique())


[4 3 0 1 2]


In [187]:
print(Data.loc[[10, 20, 30], ["What is your current academic year?", "Academic Year", "Expected Graduation Year"]])


    What is your current academic year?  Academic Year  \
10                                    4              2   
20                                    4              2   
30                                    4              2   

    Expected Graduation Year  
10                         3  
20                         3  
30                         3  


In [188]:
print(Data.dtypes)


ID                                                                 int64
First Name                                                        object
Email                                                             object
Gender                                                            object
City                                                              object
Created                                                   datetime64[ns]
Position                                                          object
New College Name                                                  object
Colleges                                                          object
Academic Year                                                      int64
Branch/ Specialisation                                            object
Other Branch                                                      object
What is your current academic year?                                int64
Company Name/ College Name                         

In [189]:
# For example, let's check the first 5 rows
for i in range(5):
    row = Data.iloc[i]
    expected_year = calculate_expected_graduation_year(row)
    print(f"Row {i+1} - Expected Graduation Year: {expected_year}")


Row 1 - Expected Graduation Year: 3
Row 2 - Expected Graduation Year: 3
Row 3 - Expected Graduation Year: 3
Row 4 - Expected Graduation Year: 3
Row 5 - Expected Graduation Year: 3


In [190]:
unique_expected_years = Data["Expected Graduation Year"].unique()
print("Unique Expected Graduation Years:", unique_expected_years)


Unique Expected Graduation Years: [3 2 1 4 5 0]


In [191]:
unique_values = Data['Academic Year'].unique()
print(unique_values)

[2 1 0 3]


In [148]:
unique_values = Data['Branch/ Specialisation'].unique()
print(unique_values)

[nan 'Information Technology (IT)' 'Other' 'Computer Science']


In [192]:
print(Data['Expected Graduation Year'])

0       3
1       3
2       3
3       3
4       3
       ..
5298    3
5299    3
5300    3
5301    4
5302    4
Name: Expected Graduation Year, Length: 5303, dtype: int64


In [194]:
# Selecting only 'ID' and 'First Name' columns
selected_columns = Data[['ID', 'First Name', 'Expected Graduation Year']]

# Save to Excel
selected_columns.to_excel('/content/cdrive/My Drive/Prediction of Placement Status Data/Year_of_Graduation.xlsx', index=False)
